In [1]:
# !pip install torchinfo

In [3]:
import torch
from PIL import Image
import torchvision.transforms as transforms

import torchvision.models as tvm
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import torch.nn as nn
%matplotlib inline
from models import dataloader



In [4]:
class Jakiro(nn.Module):
    def __init__(self):
        super(Jakiro, self).__init__()
        model_spec = tvm.efficientnet_v2_s("IMAGENET1K_V1")
        self.model_spec = nn.Sequential(model_spec.features,model_spec.avgpool)
        
        model_phase = tvm.efficientnet_v2_s("IMAGENET1K_V1")
        self.model_phase = nn.Sequential(model_phase.features,model_phase.avgpool)
        
        self.classifier = nn.Linear(1280*2,1,bias=True)
    def forward(self, spec, phase):
        spec = self.model_spec(spec)
        phase = self.model_phase(phase)
        concatenated = torch.cat((spec,phase),dim=1)
        concatenated =  torch.flatten(concatenated, 1)
        
        concatenated = self.classifier(concatenated)

        return concatenated


        
        

In [5]:
model = Jakiro()

Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:01<00:00, 49.8MB/s]


In [ ]:
train_dataset,test_dataset,train_loader,test_loader = dataloader.get_dataloader()

In [ ]:
'alberta_mall_window=196_vol=35%_gun=1.png' == 'alberta_mall_window=196_vol=35%_gun=1.png'

In [ ]:
import torch.optim as optim
opt = optim.AdamW(model.parameters(), lr=0.00001, weight_decay=1e-5)


In [ ]:
sch = optim.lr_scheduler.CosineAnnealingWarmRestarts(opt,1,1)
criterion = torch.nn.BCEWithLogitsLoss()

In [ ]:
from timm.utils import accuracy, AverageMeter

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for epoch in range(20):
    model.train()
    opt.zero_grad()
    loss_meter = AverageMeter()
    for idx, (samples,mfcc, targets) in enumerate(train_loader):
        num_steps = len(train_loader)
        samples = samples.to(device)
        mfcc = mfcc.to(device)
        
        targets = targets.to(device)
        targets = targets.view(targets.shape[0], 1).to(dtype=torch.float)

        outputs = model(samples,mfcc)
        outputs = outputs.view(outputs.shape[0], 1).to(dtype=torch.float)
        loss = criterion(outputs, targets)
        loss.backward()
        opt.step()
        loss_meter.update(loss.item(), targets.size(0))

    print(
                f'Train: [{epoch}/30][{idx}/{num_steps}]\t'
                    f'loss {loss_meter.val:.4f} ({loss_meter.avg:.4f})\t'
    )
    #Validation
    model.eval()
    with torch.no_grad():
        total_acc=0
        total_samples=0
        for idx, (val_images,val_mfcc, val_target) in enumerate(test_loader):
            val_target = val_target.to(device)
            val_images = val_images.to(device)
            val_mfcc = val_mfcc.to(device)
            
        #         target = target.view(target.shape[0], 1).to(dtype=torch.float)
    
            val_outputs = model(val_images,val_mfcc)
            val_outputs = val_outputs.squeeze()
            val_outputs = torch.sigmoid(val_outputs)
            acc = (val_target.eq((val_outputs>0.5).to(dtype=torch.long)).cpu().sum())
            # print(acc)
            total_acc+=acc
            total_samples+=val_target.shape[0]
    
        print(f'Acc: {total_acc/total_samples}\t')

        

In [11]:
model = tvm.efficientnet_v2_s('IMAGENET1K_V1')

In [12]:
 model.classifier[1] = nn.Linear(1280,1,bias=True)

In [13]:
opt = optim.AdamW(model.parameters(), lr=0.00001, weight_decay=1e-5)
sch = optim.lr_scheduler.CosineAnnealingWarmRestarts(opt,2,1)
criterion = torch.nn.BCEWithLogitsLoss()

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for epoch in range(20):
    model.train()
    opt.zero_grad()
    loss_meter = AverageMeter()
    for idx, (samples,mfcc, targets) in enumerate(train_loader):
        num_steps = len(train_loader)
        samples = samples.to(device)        
        targets = targets.to(device)
        targets = targets.view(targets.shape[0], 1).to(dtype=torch.float)

        outputs = model(samples)
        outputs = outputs.view(outputs.shape[0], 1).to(dtype=torch.float)
        loss = criterion(outputs, targets)
        loss.backward()
        opt.step()
        loss_meter.update(loss.item(), targets.size(0))

    print(
                f'Train: [{epoch}/30][{idx}/{num_steps}]\t'
                    f'loss {loss_meter.val:.4f} ({loss_meter.avg:.4f})\t'
    )
    #Validation
    model.eval()
    with torch.no_grad():
        total_acc=0
        total_samples=0
        for idx, (val_images,val_mfcc, val_target) in enumerate(test_loader):
            val_target = val_target.to(device)
            val_images = val_images.to(device)
            
        #         target = target.view(target.shape[0], 1).to(dtype=torch.float)
    
            val_outputs = model(val_images)
            val_outputs = val_outputs.squeeze()
            val_outputs = torch.sigmoid(val_outputs)
            acc = (val_target.eq((val_outputs>0.5).to(dtype=torch.long)).cpu().sum())
            # print(acc)
            total_acc+=acc
            total_samples+=val_target.shape[0]
    
        print(f'Acc: {total_acc/total_samples}\t')

        

Train: [0/30][144/145]	loss 1.1804 (0.5250)	
Acc: 0.82914137840271	
Train: [1/30][144/145]	loss 0.3081 (0.2985)	
Acc: 0.8529921770095825	
Train: [2/30][144/145]	loss 0.5784 (0.2602)	
Acc: 0.8026886582374573	
Train: [3/30][144/145]	loss 0.2315 (0.2545)	
Acc: 0.8499566316604614	
Train: [4/30][144/145]	loss 0.3352 (0.2160)	
Acc: 0.8590633273124695	
Train: [5/30][144/145]	loss 0.0649 (0.2012)	
Acc: 0.8568950295448303	
Train: [6/30][144/145]	loss 0.3942 (0.1661)	
Acc: 0.8616652488708496	
Train: [7/30][144/145]	loss 0.0201 (0.1348)	
Acc: 0.8464874029159546	
Train: [8/30][144/145]	loss 0.0125 (0.1155)	
Acc: 0.8633998036384583	
Train: [9/30][144/145]	loss 0.0688 (0.1125)	
Acc: 0.8547267913818359	
Train: [10/30][144/145]	loss 0.0422 (0.1252)	
Acc: 0.8425845503807068	
Train: [11/30][144/145]	loss 3.0765 (0.1201)	
Acc: 0.8456200957298279	
Train: [12/30][144/145]	loss 0.3364 (0.1049)	
Acc: 0.8525585532188416	
Train: [13/30][144/145]	loss 0.8659 (0.0959)	
Acc: 0.8499566316604614	
Train: [14/30][144